# Implementación paso a paso de una  red neuronal recurrente

En esta actividad, implementaremos nuestra primer red neuronal recurrente.

Las redes neuronales recurrentes son efectivas en el procesamiento del lenguaje natural y otras tareas secuenciales debido a que tienen "memoria". Las redes neuronales recurrentes pueden leer datos de entrada $x^{\langle t \rangle}$ (por ejemplo palabras) una a la vez, y recordar información/contexto a través de las activaciones entre las capas ocultas que se pasan de una a otra en cada al procesar cada elemento de la secuencia de entrada.   Esto permite que un red neuronal recurrente unidireccional pueda tomar información del pasado para procesar entradas posteriores. Adicionalmente, una red neuronal recurrente bidireccional puede considerar el contexto tanto del pasado como del futuro.

**Notación**:

- El sub-índice $i$ denota el $i$-ésimo elemento de un vector.

- El super-índice $(i)$ denota un elemento asociado con el ejemplo $i^{th}$. 

- El super-índice $\langle t \rangle$ un elemento de la secuencia en el t-ésimo instante (tiempo). 

- El super-índice $[l]$ denota un elemento asociado con la capa $l$-ésima. 


Antes de iniciar, importemos la librería que necesitaremos para esta actividad y definamos la función `softmax()` que utilizaremos posteriormente.

¿Recuardas para que hemos utilizado anteriormente la función `softmax`?

In [ ]:
import numpy as np
#from rnn_utils import *

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

## 1 - Forward propagation para una  red neuronal recurrente (básica)

La red neuronal recurrente que implementaremos tiene la siguiente estructura.

<img src="images/RNN.png" style="width:300;height:190px;">
<caption><center> Figura 1: Modelo de la red neuronal recurrente </center></caption>

Observe que para este ejemplo, $ T_x = T_y $.


Las tareas a realizar en esta actividad son las siguientes:

**Tareas**:
1. Implementar los cálculos necesarios para un time-step de la red neuronal recurrente.
2. Implementar un ciclo para $T_x$ time-steps con el fin de procesar todas las entradas, una a la vez.

***¡Iniciemos con la actividad!***


## 1.1 - Celda de la Red Neuronal Recurrente

Una red neuronal recurrente puede verse como la repetición de una celda. Implementemos los cálculos necesarios para un time-step. La siguiente figura describe el conjunto de operaciones:

<img src="images/rnn_step_forward.png" style="width:300px;height:350px;">


<caption><center> Figura 2: Celda de una red neuronal recurrente. Recibe como entrada $x^{\langle t \rangle}$ (entrada actual) y $a^{\langle t - 1\rangle}$ (activación de la capa oculta previa que contenie información del pasado), y la salida $a^{\langle t \rangle}$ que sale hacía la siguiente celda de la red neuronal y que también predice $y^{\langle t \rangle}$ </center></caption>


**Ejercicio 1**: Implementemos la celda descrita en la figura 2.

**Pasos**:
1. Calcule la activación de la capa oculta (utilizaremos la función tanh): $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.

2. Utilizando la activación de la capa oculta previa $a^{\langle t \rangle}$, calcule la predicción $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. Para esto utilicemos la función: `softmax`.

3. Guardemos en cache los valores de $a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters$.

4. La salida de la celda es: $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ y el cache.

Para la implementación, vectorizaremos las operaciones sobre $m$ ejemplos. Así, $x^{\langle t \rangle}$ tendrá la dimensión $(n_x,m)$, y $a^{\langle t \rangle}$ tendrá la dimensión $(n_a,m)$. 

In [ ]:

def cell_forward(xt, a_prev, parameters):
    """
    Argumentos:
    
    - xt           Datos de entrada para el timestep "t", es un arreglo numpy de dimensiones (n_x, m)
    
    - a_prev       Activación previa en el timestep "t-1", es un arreglo numpy de dimensiones (n_a, m)

    - parameters   Diccionario de python que contiene:
                    Wax      matrix de pesos para la entrada, es un arreglo numpy de dimensiones (n_a, n_x)
                    Waa      matrix de pesos para la activación previa, arreglo numpy de dimensiones (n_a, n_a)
                    Wya      matrix de pesos que relaciona la cap oculta con la salida, 
                             es un arreglo numpy de dimensiones (n_y, n_a)
                    ba       bias, arreglo numpy de dimensiones (n_a, 1)
                    by       bias, que relaciona la capa oculta con la salida, 
                             arreglo numpy de dimensiones (n_y, 1)
    Retorna:
    - a_next       activación de salida, de dimensiones (n_a, m)
    - yt_pred      predicción en el timestep "t", arreglo numpy de dimensiones (n_y, m)
    - cache        una tupla de valores necesarios para el backward propagation, 
                 contiene (a_next, a_prev, xt, parameters)
    """
    
    # recuperemos los elementos del diccionario "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # calculemos la activación de salida de la celda, utilizando la fórmula descrita previament
    a_next = None

    # calculemos la salida de la celda actual, utilizando la fórmula descrita previamente
    yt_pred = None 
    
    # Almacena en cache los valores que necesitaremos para el backward propagation
    cache = (None, None, None, None)
    
    return a_next, yt_pred, cache

Probemos nuestra implementación. Para esto generemos de manera aleatorio algunos datos de entrada. 

In [ ]:
np.random.seed(1)               #Definamos la semilla en 1 para poder comparar los resultados
xt = np.random.randn(3,10)      #10 ejemplos, con 3 características cada ejemplo

a_prev = np.random.randn(4,10)  # La activación tiene 4 elementos (a1, a2, a3, a4), para 10 ejemplos //
Waa = np.random.randn(4,4)      # Generá de manera aleatoria los pesos de la matriz (4, 4)
Wax = np.random.randn(4,3)      # Generá de manera aleatoria los pesos de la matriz (4, 3)
Wya = np.random.randn(2,4)      # Generá de manera aleatoria los pesos de la matriz (2, 4)
ba = np.random.randn(4,1)       # Generá de manera aleatoria el bias (4, 1)
by = np.random.randn(2,1)       # Generá de manera aleatoria el bias (2, 1), la salida tiene dos elementos (y1, y2)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}  # Almacenamos los datos en el diccionario

#Invocamos a la función cell_forward con los parámetros previamente inicializados
a_next, yt_pred, cache = None

#Verifiquemos algunos datos resultantes
print("a_next.shape = ", None)
print("a_next[2] = ", None)
print("yt_pred.shape = ", None)
print("yt_pred[0] =", None)

***Salida esperada***

`a_next.shape =  (4, 10)`

`a_next[2] =  [-0.98651884  0.67677561 -0.05660542  0.94955795 -0.8008664   0.38586318
 -0.88278509  0.8682936  -0.50437243  0.38364728]`

`yt_pred.shape =  (2, 10)`

`yt_pred[0] = [0.03454302 0.12373132 0.04064155 0.17177199 0.03098326 0.07658893
 0.05752428 0.26866085 0.21528314 0.395644  ]`

## 1.2 - Forward propagation

Una red neuronal recurrente se puede ver como la repetición de la celda que acabamos de implementar. Si la secuencia de entrada tiene 10 elementos, entonces la celda se debe replicar 10 veces. Cada celda recibe como entrada la activación de la celda (capa) oculta previa ($a^{\langle t-1 \rangle}$) y el dato de entrada del time-step actual ($x^{\langle t \rangle}$). La celda actual, generá como salida una activación ($a^{\langle t \rangle}$) y una predicción ($y^{\langle t \rangle}$) para su time-step.

<img src="images/rnn.png" style="width:800px;height:300px;">
<caption><center> Figure 3: Red neuronal recurrente. La secuencia de entrada $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  se procesa mediante $T_x$ time-steps. Las salidas de la red son $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>


**Exercicio 2**: Implementemos el forward propagation de la red neuronal describa en la Figura 3.

**Pasos**:
1. Crear un vector inicializado en cero ($a$) que almacenará todas las activaciones calculadas por la red neuronal.
2. Inicializar el estado de la activación $a_0$ (activación inicial)
3. Iniciar un ciclo sobre cada time-step, el indice que imcrementará es $t$:
    - Actualiza la siguiente activación y el cache ejecutando `cell_forward`
    - Almacena la activación en $a$ ($t$-ésima posición) 
    - Almacena la predicción en `y`
    - Agrega el cache a la lista de caches.
4. Retorna $a$, $y$ y los caches

In [ ]:

def network_forward(x, a0, parameters):
    """
    Implementemos el forward propagation de la red neuronal recurrente descrita en la figura 3
    
    Argumentos:
    
    x             Datos de entrada para cada time-step, de dimensiones (n_x, m, T_x).
    
    a0            Activación incial, de dimensiones (n_a, m)

    parameters    Diccionario de python con:
        Waa    matriz de pesos para la función de activación, arreglo numpy de dimensiones (n_a, n_a)
        Wax    matriz de pesos para datos de entra, arreglo numpy de dimensiones (n_a, n_x)
        Wya    matriz de pesos relacionando la activación con la salida, arreglo numpy de dimensiones (n_y, n_a)
        ba     bias para activación, arreglo numpy de dimensiones (n_a, 1)
        by     bias para la salida, arreglo numpy de dimensiones (n_y, 1)

    Retorna:
    a             activaciones de los time-step, arreglo numpy de dimensiones (n_a, m, T_x)
    yp            predicciones para cada time-step, arreglo numpy de dimensiones (n_y, m, T_x)
    caches        tupla de valores requeridos para el backward propagation. Contiene (lista de caches, x)
    """
    
    # Inicializa los "caches" que contendrá la lista de todos los caches
    caches = []
    
    # Recupera las dimensiones de "x", y  parameters["Wya"]
    n_x, m, T_x = None
    n_y, n_a = None
        
    # Inicializa "a", y "yp" en ceros
    a = np.zeros([n_a,m,T_x])
    yp = np.zeros([n_y,m,T_x])
    
    # Inicializa la siguiente activación: a_next 
    a_next = a0
    
    # Itera sobre todos los time-steps
    for t in range(T_x):
        
        # Actualiza la activación, calcula la predicción, tomar el cache
        # invocando la función cell_forward
        
        a_next, yp_t, cache = cell_forward(x[:,:,t], None, None)
        
        # Almacena el valor de la siguiente activación en a
        a[:,:,t] = None
        
        # Almacena el valor de la predicción en yp 
        yp[:,:,t] = None
        
        # Agregar a la lista caches el cache actual 
        caches.append(None)
        
    
    # Almacena los valores necesarios para el backward propagation en cache
    caches = (None, None)
    
    return a, yp, caches

Probemos nuestra implementación. Para esto generemos de manera aleatorio algunos datos de entrada. 

In [ ]:
np.random.seed(1)               # Se define una semilla para comparar los resultados
x = np.random.randn(3,10,4)     # Generá de manera aleatoria los datos de entrada, 4 secuencias, longitud=10, cada elemento tiene 3 características
a0 = np.random.randn(4,10)      # Inicializa de manera aleatoria la activación inicial (4, 10)
Waa = np.random.randn(4,4)      # Generá de manera aleatoria los pesos de la matriz (4, 4)
Wax = np.random.randn(4,3)      # Generá de manera aleatoria los pesos de la matriz (4, 3)
Wya = np.random.randn(2,4)      # Generá de manera aleatoria los pesos de la matriz (2, 4)
ba = np.random.randn(4,1)       # Generá de manera aleatoria el bias (4, 1)
by = np.random.randn(2,1)       # Generá de manera aleatoria el bias (2, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

# Invoquemos la función network_forward con los parámetros previamente definidos
a, y_pred, caches = network_forward(None, None, None)

# Verifiquemos algunos datos resultantes
print("len(caches) = ", len(None))
print("a.shape = ", None)
print("a[2][1] = ", None)
print("y_pred.shape = ", None)
print("y_pred[1][2] =", None)
print("caches[1][1][2] =", None)

***Salida esperada:***

`len(caches) =  2`

`a.shape =  (4, 10, 4)`

`a[2][1] =  [-0.88357333  0.97830926 -0.92193859  0.99991081]`

`y_pred.shape =  (2, 10, 4)`

`y_pred[1][2] = [0.79621528 0.25637256 0.99608341 0.18712976]`

`caches[1][1][2] = [ 0.12015895  0.61720311  0.30017032 -0.35224985]`

Hasta este momento hemos creado la etapa de Forward Propagation de la red neuronal recurrente. Esto funcionará lo suficientemente bien para algunas aplicaciones, pero adolece de problemas de gradiente que se desvanecen. Por lo tanto, funciona mejor cuando cada salida $y^{< t >} $ se puede estimar utilizando principalmente el contexto "local" (es decir, la información de las entradas $x^{\langle t' \rangle} $ donde $t'$ no se encuentra muy distante de $t$).